# Importing Libraries
***

In [1]:
from neo4j import GraphDatabase
from neo4j.graph import Node, Relationship
from neo4j.data import Record
from neo4j import unit_of_work
from pandas import DataFrame
import pandas as pd
import numpy as np


C:\Users\odyky\AppData\Local\Temp\ipykernel_17608\3505887652.py:3: DeprecationWarning: The module 'neo4j.data' was made internal and will no longer be available for import in future versions. `neo4j.data.Record` should be imported directly from `neo4j`.
  from neo4j.data import Record


# Connecting to Neo4j
***

In [3]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "123123123"))

# Deffining Useful Functions
***

In [2]:
def run_query(query): 
     with driver.session() as session:
        result = session.run(query)
        #print(result.single()[0])
        
        
def delete_all_nodes():
    query = '''
    MATCH(n) DETACH DELETE(n)
    '''
    run_query(query)

def record_count():
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) LIMIT 100")
        print(result.single())

def delete_nodes(var):
    query = f'''
    MATCH (c:{var})
    DELETE c

    '''
    run_query(query)

***
# Graph Algorithms

## Algorithm No. 1 - Path Finding Algorithms: Dijkstras shortest path

### Calculating the shortest path between two Papers in terms of citations

### Creating the projection for Dijkstras shortest path algorithm with gds library

In [7]:
query = '''CALL gds.graph.project.cypher(
  'paper_cites',
  'MATCH (p:Paper) RETURN id(p) AS id',
  'MATCH (p:Paper)-[c:cites]->(p2:Paper) RETURN id(p) AS source, id(p2) AS target, type(c) as type')'''

run_query(query)

### Executing Dijkstras shortest path algorithm from gds library
#### Papers *Undirected graphs of entanglement 2* and *Fingerprint recognition based on minutes groups using directing attention algorithms* are being used

In [8]:
title1 = 'Undirected graphs of entanglement 2'
title2 = 'Fingerprint recognition based on minutes groups using directing attention algorithms'


with driver.session() as session:
    result = session.run('''
    MATCH (p:Paper {title: $titol1})
    WITH id(p) as id_P, p
    MATCH (p2:Paper {title: $titol2})
    WITH id(p) as id_P, p, id(p2) as id_P2, p2

    CALL gds.allShortestPaths.dijkstra.stream('paper_cites', {
        sourceNode: p
    })

    YIELD sourceNode, targetNode, totalCost
    WHERE targetNode = id_P2
    RETURN gds.util.asNode(sourceNode).title as sourceNode, gds.util.asNode(targetNode).title as targetNode, totalCost
    ''', titol1 = title1, titol2 = title2)

    res = []
    for record in result:
        source_node = record['sourceNode']
        target_node = record['targetNode']
        cost = record['totalCost']

        res.append({'SourceNode': source_node, 'TargetNode': target_node, "Cost": cost})
        df = pd.DataFrame.from_records(res)

df

,SourceNode,TargetNode,Cost
0,Undirected graphs of entanglement 2,Fingerprint recognition based on minutes group...,5.0


## Algorithm No. 2 - Similarity Algorithms: Node similarity

### Calculate similarity between Papers based on their shared keywords

### Creating the projection for Node similarity algorithm with gds library

In [5]:
query = '''CALL gds.graph.project.cypher(
  'papers_and_keywords',
  'MATCH (a) WHERE a:Paper OR a:Keyword RETURN id(a) AS id, labels(a) AS labels',
  'MATCH (a:Paper)-[r:has_keyword]->(k: Keyword) RETURN id(a) AS source, id(k) AS target, type(r) as type')
  YIELD
  graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipCount AS rels'''

run_query(query)

### Executing Node similarity algorithm from gds library

In [6]:
with driver.session() as session:
    result = session.run('''
    CALL gds.nodeSimilarity.stream('papers_and_keywords', { degreeCutoff: 5, similarityCutoff: 0.5})
    YIELD node1, node2, similarity
    RETURN gds.util.asNode(node1).title AS Paper1, gds.util.asNode(node2).title AS Paper2, similarity
    ORDER BY similarity DESCENDING, Paper1, Paper2
    ''')

    res = []
    for record in result:
        source_node = record['Paper1']
        target_node = record['Paper2']
        similarity = record['similarity']

        res.append({'SourceNode': source_node, 'TargetNode': target_node, "Similarity": similarity})
        df = pd.DataFrame.from_records(res)

df

,SourceNode,TargetNode,Similarity
0,A new index of creditworthiness for retail cre...,Testing schedule performance and reliability f...,1.000000
1,Testing schedule performance and reliability f...,A new index of creditworthiness for retail cre...,1.000000
2,A new index of creditworthiness for retail cre...,"Optimal policies for inventory usage, producti...",0.944444
3,"Optimal policies for inventory usage, producti...",A new index of creditworthiness for retail cre...,0.944444
4,"Optimal policies for inventory usage, producti...",Testing schedule performance and reliability f...,0.944444
5,Testing schedule performance and reliability f...,"Optimal policies for inventory usage, producti...",0.944444
6,Ciruvis: a web-based tool for rule networks an...,MetRxn: a knowledgebase of metabolites and rea...,0.666667
7,MetRxn: a knowledgebase of metabolites and rea...,Ciruvis: a web-based tool for rule networks an...,0.666667
8,Massive MIMO Multicasting in Noncooperative Ce...,Massive MIMO With Joint Power Control,0.555556
9,Massive MIMO With Joint Power Control,Massive MIMO Multicasting in Noncooperative Ce...,0.555556
